# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

Das ist eine der populärsten Use Cases von Langchain. Wir werden eine Beauskunftung über ein Dokument machen, das nicht zum Training des Modells verwendet wurde. 

Hierzu werden wir Vektorembeddings und Vektordatenbank verwenden und damit zu den avancierteren Bausteinen von langchain übergehen.

In [11]:
pip install --upgrade langchain

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/7a/7d/d7be01f6f02a1a7720b8953cfec14f945b02001842516c92c4169daf99f3/langchain-0.0.322-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.4 MB/s eta 0:00:0000:01
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.321
    Uninstalling langchain-0.0.321:
      Successfully uninstalled langchain-0.0.321

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

Hier nutzen wir eine leichtgewichtige Vektordatenbank, die im Arbeitsspeicher lebt und daher keine gesonderte Installation erfordert.

In [3]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [15]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [5]:
from langchain.indexes import VectorstoreIndexCreator

In [6]:
pip install docarray

  Obtaining dependency information for docarray from https://files.pythonhosted.org/packages/1e/50/89ebe5de8189fa049f3a36a1a2151a72de27ccddbb3786eeeee75388ab64/docarray-0.39.1-py3-none-any.whl.metadata
  Obtaining dependency information for orjson>=3.8.2 from https://files.pythonhosted.org/packages/f4/66/d13cb488f5e74d603ebeb0b7b5032a81462d76d0b197a7b2002843ca1055/orjson-3.9.9-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.4 MB/s eta 0:00:00
  Obtaining dependency information for rich>=13.1.0 from https://files.pythonhosted.org/packages/be/2a/4e62ff633612f746f88618852a626bbe24226eba5e7ac90e91dcfd6a414e/rich-13.6.0-py3-none-any.whl.metadata
  Obtaining dependency information for types-requests>=2.28.11.6 from https://files.pythonhosted.org/packages/1d/d6/1281c1d7b03a127562d6644ebff081e85045f0025b1fe26dcdd82811ad1a/types_requests-2.31.0.10-py3-none-any.whl.metadata
  Obtaining dependency information for mar

In [16]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [17]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [18]:
response = index.query(query)

ValidationError: 2 validation errors for DocArrayDoc
text
  Field required [type=missing, input_value={'embedding': [0.00331889..., -0.02122992287370924]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.4/v/missing
metadata
  Field required [type=missing, input_value={'embedding': [0.00331889..., -0.02122992287370924]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.4/v/missing

In [19]:
display(Markdown(response))

NameError: name 'response' is not defined

## Step By Step

In [ ]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file)

In [ ]:
docs = loader.load()

In [ ]:
docs[0]

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [ ]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [ ]:
print(len(embed))

In [ ]:
print(embed[:5])

In [ ]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [ ]:
query = "Please suggest a shirt with sunblocking"

In [ ]:
docs = db.similarity_search(query)

In [ ]:
len(docs)

In [ ]:
docs[0]

In [ ]:
retriever = db.as_retriever()

In [ ]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

In [ ]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [ ]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


In [ ]:
display(Markdown(response))

In [ ]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [ ]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [ ]:
response = qa_stuff.run(query)

In [ ]:
display(Markdown(response))

In [ ]:
response = index.query(query, llm=llm)

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

Reminder: Download your notebook to you local computer to save your work.